# TRIQS Green's functions

It is now time to start using some of the tools provided by TRIQS.

Much of the functionality in TRIQS, while implemented in C++ for optimal performance, is exposed
through a Python interface to make it easier to use. From a practical point of view this means
that you can think of TRIQS as a python library, just like numpy or matplotlib.

One of the central objects of a many-body calculation is a Green's function.
Green functions in TRIQS are functions defined on a mesh $\cal{M}$ of points that hold values in some domain $\cal{D}$, for example $\mathbb{C}^{2\times2}$

$$
G: \cal{M} \rightarrow \cal{D}
$$

A few common Green function meshes in TRIQS include:

- `MeshReFreq` - Real-frequencies equally spaced in $[\omega_{min},\omega_{max}]$
- `MeshImFreq` - Matsubara Frequencies
- `MeshImTime` - Imaginary time points equally spaced in $[0,\beta]$
- `MeshReTime` - Real-time points (not covered in this tutorial)

Let's see how we can **construct a Mesh and print its values**.

In [ ]:
# Import the Mesh type we want to use
from triqs.gf import MeshImTime

# The documentation tells us which parameters we need to pass for the mesh construction
?MeshImTime

In [ ]:
# Provide the inverse temperature, Statistic, and number of points
tau_mesh = MeshImTime(beta=5, S='Fermion', n_tau=11)

# We can loop and print the mesh-point values
for tau in tau_mesh:
    print(tau.value)

Let us now **create and initialize a Green function for a single atomic level** with energy $\epsilon$ in the grand-canonical ensemble with inverse temperature $\beta$

$$
G[\tau] = -\langle\cal{T}c(\tau) c^\dagger\rangle = \frac{e^{-\tau \epsilon}}{1+e^{-\beta \epsilon}}
$$

We first have a look at the documentation for `Gf`.

In [ ]:
from triqs.gf import Gf
?Gf

In [ ]:
# Create scalar-valued imaginary-time Green function
G = Gf(mesh=tau_mesh, target_shape=[], is_real=True)

# Print the Green function description
print(G)

# Loop initialization
eps = -0.4
beta = G.mesh.beta
from math import exp
for tau in G.mesh:
    # Slightly adjusted expression for numerical stability
    G[tau] = -exp((beta*(eps<0) - tau.value) * eps) / (1. + exp(-beta * abs(eps)))
    print("{:.3f}".format(G[tau]))

In order to **plot this Green function** we can use the matplotlib interface defined in TRIQS.
Note that the function to plot Green's function is `oplot` and not just `plot` like in matplotlib.

In [ ]:
from triqs.plot.mpl_interface import oplot,plt

# Make plots show up directly in the notebook:
%matplotlib inline

# Make all figures slightly bigger
import matplotlib as mpl
mpl.rcParams['figure.dpi']=100

# Additional arguments like 'linewidth' are passed on to matplotlib
oplot(G, '-', name='G', linewidth=2)

## Matrix-Valued Green functions

In most realistic problems we have to treat more than just a single orbital

$$
G_{ij}[\tau] = -\langle\cal{T}c_i(\tau) c_j^\dagger\rangle
$$

For this purpose, TRIQS provides Green functions that have a Matrix structure. Let's see how you can create and use them

In [ ]:
# A uniform real-frequency mesh on a given interval
from triqs.gf import MeshReFreq
w_mesh = MeshReFreq(omega_min=-4, omega_max=4, n_w=1000)

# Gf with 2x2 Matrix structure holding complex values
G = Gf(mesh=w_mesh, target_shape=[2,2])
G # <- Same as print(repr(G))

In [ ]:
# Accessing a specific mesh point gives us a matrix
from triqs.gf import Idx # Use Idx to access Gf at specific Index
print(G[Idx(0)])

In [ ]:
# By Fixing the orbital indices we obtain a Green function that is no longer matrix but complex-valued
G[0,0]

## Block Green functions

In many realistic problems we know a priori that (due to e.g. symmetries or conserved quantum numbers) certain components of the Green function will vanish.
In other words, the Green's functions has an additional *block structure*.

$$
\hat{G} =
\begin{pmatrix}
\hat{g}^0 & 0 & \cdots & \cdots \\
0 & \hat{g}^1 & 0 & \cdots \\
\cdots & 0 & \hat{g}^2 & 0 \\
\cdots & \cdots & \cdots & \cdots
\end{pmatrix}
$$

Here the $\hat{g}^i$ are Green's functions with non-zero elements $g^i_{ab}$. In principle they can have
different dimensions.

For example, you can imagine a system of 5 $d$-orbitals that are split by a
crystal field into 3 $t_{2g}$-orbitals and 2 $e_g$-orbitals. For symmetry reasons, you can have a
situation where these orbitals do not talk to each other. In that case, the complete Green's function
would have two blocks, one of size 2x2 corresponding to the $e_g$ orbitals and one of size 3x3 corresponding for the $t_{2g}$ orbitals. 

$$
\hat{G} =
\begin{pmatrix}
\hat{g}^{e_g} & 0  \\
0 & \hat{g}^{t_{2g}}  \\
\end{pmatrix}=
\begin{pmatrix}
\begin{pmatrix}
g^{e_g}_{00} & g^{e_g}_{01} \\
g^{e_g}_{10} & g^{e_g}_{11}
\end{pmatrix} & 0 \\
0 & \begin{pmatrix}
g^{t_{2g}}_{00} & g^{t_{2g}}_{01} & g^{t_{2g}}_{02} \\
g^{t_{2g}}_{10} & g^{t_{2g}}_{11} & g^{t_{2g}}_{12} \\
g^{t_{2g}}_{20} & g^{t_{2g}}_{21} & g^{t_{2g}}_{22} \\
\end{pmatrix}
\end{pmatrix}
$$

Let us now consider a more concrete example for the case outlined above:
$$
\hat{G} =
\begin{pmatrix}
\begin{pmatrix}
\omega & V_1 \\
V_1 & \omega
\end{pmatrix}^{-1} & 0 \\
0 & \begin{pmatrix}
\omega - \epsilon_2 & 0 & V_2 \\
0 & \omega - \epsilon_2 & 0 \\
V_2 & 0 & \omega - \epsilon_2
\end{pmatrix}^{-1}
\end{pmatrix}
$$

The associated type in TRIQS is called `BlockGf`. Let us have a first look at it's documentation:

In [ ]:
from triqs.gf import BlockGf
?BlockGf

We will in the following consider the first two options for the construction listed in the documentation.

The first way is to simply define the two Green function blocks separately, and to then pass these blocks, together with their names:

In [ ]:
# Construct individual blocks
g_eg  = Gf(mesh=w_mesh, target_shape=[2,2])
g_t2g = Gf(mesh=w_mesh, target_shape=[3,3])

# Combine blocks into a BlockGf
G = BlockGf(name_list=['eg', 't2g'], block_list=[g_eg, g_t2g])
print(G)

We can then access individual blocks simply by using their name

In [ ]:
G['eg']

For cases where all blocks have a square-matrix `target_shape` we can alternatively pass a list of pairs of block-names and linear matrix sizes.

In [ ]:
# List of Block-names and their linear matrix size
gf_struct = [('eg',2), ('t2g',3)]

G = BlockGf(mesh=w_mesh, gf_struct=gf_struct)
print(G)

Let us now initialize the Green function values. Instead of writing a loop over the mesh, we here make use of the the operator `<<`, which can fill the Green's function with simple expressions containing the `Omega` descriptor. Here `Omega` is replaced by the values of the mesh.

In [ ]:
from triqs.gf import Omega

V1 = 0.1
V2 = 0.2
eps_t2g = -2.0

# The e_g part
G['eg'][0,0] << Omega
G['eg'][0,1] << V1
G['eg'][1,0] << V1
G['eg'][1,1] << Omega

# Perform an in-place Matrix inversion
G['eg'].invert()

# The t_2g part
G['t2g'][0,0] << Omega - eps_t2g
G['t2g'][1,1] << Omega - eps_t2g
G['t2g'][2,2] << Omega - eps_t2g
G['t2g'][0,2] << V2
G['t2g'][2,0] << V2
G['t2g'].invert()

When using the Green function object, it is often convenient to **iterate over all the blocks** of a `BlockGf`. We can do this with the following construct

In [ ]:
# Loop over the blocks
for name, g in G:
    print("This is the block called", name)
    print("The associated Green's function is", g)

## Additional Initialization Descriptors

In the following we will introduce a few additional means of initializing Green functions using `<<`.

### Flat density of states
Consider the problem of a single impurity level embedded in a flat conduction bath $\Gamma$ of electrons.
$$
g^\mathrm{imp} (\omega) = \frac{1}{\omega - \epsilon_d - V^2 \Gamma(\omega)}
$$

In the equation above $\epsilon_d$ is the energy of the level and $\Gamma$ is the Green's function of
a flat conduction bath

$$
\Gamma(\omega) = \int_{-D}^{D}\frac{1}{\omega-\epsilon + i\eta}\frac{d\epsilon}{2D}
$$

Here $D$ denotes the half-bandwidth.
Let's see how to define and then plot this Green's function by using `inverse` and the `Flat` descriptor.

In [ ]:
eps_d = 1.0 # Energy
V = 0.2   # Bath Hybridization
D = 1.5   # Half bandwidth

G = Gf(mesh=w_mesh, target_shape=[])

from triqs.gf import Omega, Flat, inverse
G << inverse(Omega - eps_d - V**2 * Flat(1.5))

Note the predefined function `Flat` for a flat conduction bath $\Gamma(\omega)$.
Let's plot the impurity Green's function. Note that default, both the real and imaginary parts are plotted.

In [ ]:
oplot(G, '-', linewidth=2, name="G") 

We can plot the spectral function, which is defined as

$$ \rho(\omega) = -\frac{1}{\pi} \, \textbf{Im} \, G $$

In [ ]:
from math import pi
oplot(-G.imag/pi, linewidth=2, name=r"$\rho$")

As expected the spectral function is peaked at $\epsilon_d$ and shows a jump in spectral weight at $D$.

### Semi-circular density of states

Another predefined Green's function is the one corresponding to a semi-circular spectral function. This one will be useful in the DMFT Tutorials later on.

In [ ]:
D = 1.0 # Half bandwidth

G = Gf(mesh=w_mesh, target_shape=[])

from triqs.gf import SemiCircular
G << SemiCircular(D)

oplot(-G.imag/pi, name=r"$\rho$") 

Imaginary-frequency Green's functions
-------------------------------------

These are Green's function defined on the Matsubara axis. The fermionic Matsubara frequencies
are defined by

$$\omega_n = \frac{(2n+1)\pi}{\beta}$$

where $\beta = 1/T$ is the inverse temperature. These Green's functions are important because
most Monte Carlo algorithms yield results on the Matsubara axis. Let's see how they
are defined:

In [ ]:
# Define the imaginary-frequency mesh
from triqs.gf import MeshImFreq
iw_mesh = MeshImFreq(beta=5, S='Fermion', n_iw=1000)

# Create Green Function and fill it using the iOmega_n descriptor
G = Gf(mesh=iw_mesh, target_shape=[])
from triqs.gf import iOmega_n
G << inverse(iOmega_n - 0.2)

# Plot the Green's function
oplot(G, '-o', name='G')
plt.xlim(0,10)

## Arithmetic Operations

Green's functions can be added, multiplied by numbers, etc. The way this is done is quite natural.

In [ ]:
oplot(G, "-o", name='G')
oplot(G+G, "-o", name='G+G')
oplot(3*G+2, "-o", name='3*G+1')
plt.xlim(0,10)

## Obtaining the density

You can obtain the density for Green functions with a `MeshReFreq` and `MeshImFreq` using the `density` method

In [ ]:
G = Gf(mesh=iw_mesh, target_shape=[])
G << inverse(iOmega_n - 0.2)
print("Density =", G.density())

Do not worry about the imaginary component as the machine precision is on the order of $10^{-15}$.

## Fourier transforms

TRIQS allows you to easily Fourier transform Green's functions from imaginary-time to imaginary-frequency.

In [ ]:
# A Green's function in frequency set to semi-circular
Giw = Gf(mesh=iw_mesh, target_shape=[])
Giw << SemiCircular(1.0)

# A Green's function in time set by inverse Fourier transform
from triqs.gf import make_gf_from_fourier
Gtau = make_gf_from_fourier(Giw)
oplot(Gtau, name='G')

We can also go the other way. Let's check that it gives back the original result.

In [ ]:
Giw_2 = make_gf_from_fourier(Gtau)
oplot(Giw, 'o')
oplot(Giw_2, 'x')
plt.xlim(0,5)

In the example above `make_gf_from_fourier` will construct a new Green Function object.
Alternatively we can initialize an existing Green function using the `Fourier` descriptor

In [ ]:
from triqs.gf import Fourier
Gtau << Fourier(Giw)

## Pade analytical continuation

The Fourier transforms allow to go from time to frequency. A much more delicate thing is to do the so-called "analytical continuation". This means to start from a Matsubara-frequency Green's function and obtain the corresponding real-frequency Green's function. This can formally be done, but turns out to be a mathematically ill-conditioned problem. Even small amounts of noise in the Matsubara-frequency data will make the continuation to the real axis very unstable.

One of the ways to do perform analytical continuation is to use [Pade approximants](https://en.wikipedia.org/wiki/Padé_approximant#Definition). TRIQS can do that for you in the following way:

*Note:* Pade is currently implemented only for Green Functions with a Matrix structure

In [ ]:
# The Matsubara Green's function to be continued
iw_mesh = MeshImFreq(beta=50, S='Fermion', n_iw=1000)
Giw = Gf(mesh=iw_mesh, target_shape=[1,1])
Giw << SemiCircular(1.0)

# Construct real-frequency Green function and initialize it using Pade approximants
Gw = Gf(mesh=w_mesh, target_shape=[1,1])
Gw.set_from_pade(Giw)

oplot(-Gw.imag/pi, linewidth=2)

The coarse Matsubara discretization at high temperatures will worsen the Pade result, which is why we chose a much lower temperature value for this example.

You can see that the Pade continuation did a pretty good job. We will see later that noise will completely change this picture!

## Exercises

### Exercise 1

Define the following real-frequency Green's function, where $\Gamma$ is the Green's function of a
flat bath (width = 1), $\epsilon_d = 0.3$ and $V=0.2$:

$$
G^\mathrm{s+d} (\omega) =
\begin{pmatrix} \omega - \epsilon_d & V \\\\ V & \Gamma^{-1}
\end{pmatrix}^{-1}
$$

Plot the spectral function for both diagonal components of this Green's function. What
do they represent physically?

### Exercise 2

Plot the density $n(\epsilon)$ as a function of $\epsilon$ for a Green's function $G = 1/(i\omega_n - \epsilon)$. What is the curve that you obtained? How does it change with temperature?

### Exercise 3

Define a block Green's function with an *up* and a *down* block. Each block is just a simple 1x1 imaginary-frequency Green's function. Iterate over the blocks to initialize the two blocks to $1/i \omega_n$. What happens if you change $\beta$?

### Exercise 4

Consider a Hubbard atom with $U=2$ at temperature $T = 1/\beta = 1/10$. The non-interacting and interacting Green's functions for this problem are:

$$
G_0 = \frac{1}{i \omega_n + U/2} \qquad G = \frac{1}{2(i\omega_n + U/2)} + \frac{1}{2(i\omega_n - U/2)}
$$

Using Dyson's equation, verify that the corresponding self-energy is indeed

$$
\Sigma = \frac{U}{2} + \frac{U^2}{4 i\omega_n}
$$

### Exercise 5

Compute the following second-order self-energy with $U=2$ and $\beta=50$

$$ \Sigma(i\omega_n) = U^2 \int_0^\beta d\tau e^{i \omega_n \tau} G_0(\tau)^3 $$

using an non-interacting $G_0$ given by a semi-circular of half-bandwidth 1. Use Dyson's equation to compute $G(i\omega_n)$.

Hint: The `SemiCircular` initializer only works for frequency Green's functions.

Hint: The "power operator" is not defined for Green's functions. Use multiplication.

### Exercise 6

Use Pade approximants to obtain a real-frequency version of the Green's function computed in the Exercise 5. What is the effect of interactions at second-order perturbation theory? How is it changing with different values of $U$?